In [3]:
# Importação das bibliotecas necessárias

import pandas as pd
import numpy as np
import glob
import os
import re
import psycopg2
from datetime import datetime
import locale

In [4]:
# Estabelecimento de conexão com o banco de dados
# para extração das informações das tabelas necessárias

connection = psycopg2.connect(
        user="cat",
        password="5pM2h0MBQu9JHkxHud2A",
        host="177.11.49.194",
        port="3361",
        database="cat"
    )

In [5]:
# Definição da loja a ser utilizada

loja = 28

In [6]:
# Importação das planilhas necessárias para
# gerar os documentos da 000, 0150, 0200, 1050 e 1100

planilha1 = pd.read_excel(f'Loja_{loja}/ficha_3_loja_{loja}_p1_v1.xlsb', sheet_name='ficha3', engine='pyxlsb')
planilha2 = pd.read_excel(f'Loja_{loja}/ficha_3_loja_{loja}_p2_v1.xlsb', sheet_name='ficha3', engine='pyxlsb')

planilha = pd.concat([planilha1, planilha2])
planilha = planilha.reset_index()
planilha['DATA'] = pd.to_datetime(planilha['DATA'], format='%Y-%m-%d')
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,VLR_CONFR_PCAT,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,ALIQUOTA,VALOR,Valor Base Cálculo ICMS ST Retido Operação Anterior,Valor Complementar,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,3,0.0,...,NaN,NaN,NaN,18.0,46.35,51.03,52.99,0.0,0.0,18.7236
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,2,NaN,...,NaN,NaN,NaN,18.0,30.90,34.02,35.33,0.0,0.0,12.4830
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,1,NaN,...,6.244000,NaN,NaN,18.0,19.72,0.00,NaN,0.0,0.0,0.0000
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,1,NaN,...,6.245000,NaN,NaN,18.0,19.72,0.00,NaN,0.0,0.0,0.0000
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,1,NaN,...,6.246667,NaN,NaN,18.0,19.72,0.00,NaN,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,575708,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,1,NaN,...,17.871644,NaN,NaN,12.0,35.47,0.00,NaN,0.0,0.0,0.0000
1337658,575709,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,1,NaN,...,17.871655,NaN,NaN,12.0,35.47,0.00,NaN,0.0,0.0,0.0000
1337659,575710,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,1,NaN,...,17.871667,NaN,NaN,12.0,45.60,0.00,NaN,0.0,0.0,0.0000
1337660,575711,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,1,NaN,...,17.871678,NaN,NaN,12.0,45.60,0.00,NaN,0.0,0.0,0.0000


In [7]:
tabela_2 = pd.read_csv(f'Loja_{loja}/loja_{loja}_tabela_2_v1.csv')
tabela_2 = tabela_2.sort_values(by=['COD_ITEM', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                           ascending=[True, True, True, False])
tabela_2

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\2603824844.py:1: DtypeWarning: Columns (20,26) have mixed types. Specify dtype option on import or set low_memory=False.
  tabela_2 = pd.read_csv(f'Loja_{loja}/loja_{loja}_tabela_2_v1.csv')


,Unnamed: 0,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_EFD,...,Valor ICMS Operação,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Valor Base Cálculo ICMS Substituição Tributária,FONTE
19554,50180,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,3,3.0,...,0.0,DEPAKENE 250MG 25'S ABB C1,7891158000942,UN,30049099.0,1300100.0,2.784996e+13,6.161357e+12,NaN,entrada_sp
20711,51428,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,2,2.0,...,0.0,DEPAKENE 250MG 25'S ABB C1,7891158000942,UN,30049099.0,1300100.0,2.784996e+13,6.161357e+12,NaN,entrada_sp
850132,997644,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,1,NaN,...,0.0,3759 - DEPAKENE 250MG 25'S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
850133,997645,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,1,NaN,...,0.0,3759 - DEPAKENE 250MG 25'S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
850134,997646,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,1,NaN,...,0.0,3759 - DEPAKENE 250MG 25'S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278870,1520082,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,1,NaN,...,0.0,82296 - ATORVASTATINA 40MG 30S EMS,7896004729442.0,UN,30049069.0,NaN,NaN,NaN,NaN,dfe
1278871,1520083,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,1,NaN,...,0.0,82296 - ATORVASTATINA 40MG 30S EMS,7896004729442.0,UN,30049069.0,NaN,NaN,NaN,NaN,dfe
1082981,1278198,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,1,NaN,...,0.0,4684 - ATORVASTATINA 40MG 3,7896004729442.0,UN,30049069.0,NaN,NaN,NaN,NaN,dfe
1082982,1278199,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,1,NaN,...,0.0,4684 - ATORVASTATINA 40MG 3,7896004729442.0,UN,30049069.0,NaN,NaN,NaN,NaN,dfe


In [8]:
planilha['DESCRICAO'] = tabela_2['DESCRICAO']
planilha['CODIGO_BARRA'] = tabela_2['CODIGO_BARRA']
planilha['UNIDADE'] = tabela_2['UNIDADE']
planilha['N C M'] = tabela_2['N C M']
planilha['CEST'] = tabela_2['CEST']
planilha['CNPJ EMITENTE'] = tabela_2['CNPJ EMITENTE']
planilha['CNPJ DESTINATARIO'] = tabela_2['CNPJ DESTINATARIO']

In [9]:
planilha.head()

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,3,0.0,...,0.0,0.0,18.7236,Clor Oxibutinina Ems 120ml Xpe,SEM GTIN,UN,30049099.0,1300201.0,6.161357e+12,0.0
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,2,NaN,...,0.0,0.0,12.4830,Aradois 25mg 30S Biolab Sanus,SEM GTIN,UN,30049069.0,1300300.0,6.161357e+12,0.0
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,1,NaN,...,0.0,0.0,0.0000,Aciclovir Sand 400mg 30S,SEM GTIN,UN,30049069.0,1300200.0,6.161357e+12,0.0
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,1,NaN,...,0.0,0.0,0.0000,Ivermectina 6mg 4S Vitamedic,SEM GTIN,UN,30049059.0,1300200.0,6.161357e+12,0.0
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,1,NaN,...,0.0,0.0,0.0000,Esc D Condor Lilica+Gel D,SEM GTIN,UN,96032100.0,2005800.0,6.161357e+12,0.0


In [10]:
planilha['DESCRICAO'] = planilha['DESCRICAO'].str.replace('"', "'")

In [11]:
planilha['N C M'] = pd.to_numeric(planilha['N C M'])
planilha['N C M'] = planilha['N C M'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CEST'] = pd.to_numeric(planilha['CEST'])
planilha['CEST'] = planilha['CEST'].astype('Int64').astype(str).replace('<NA>', np.nan)
planilha['CEST'] = planilha['CEST'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CNPJ EMITENTE'] = pd.to_numeric(planilha['CNPJ EMITENTE'])
planilha['CNPJ EMITENTE'] = planilha['CNPJ EMITENTE'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['QTD_CAT'] = planilha['QTD_CAT'].astype(str) + ',000'

planilha['COD_LEGAL_PCAT'] = pd.to_numeric(planilha['COD_LEGAL_PCAT'])
planilha['COD_LEGAL_PCAT'] = planilha['COD_LEGAL_PCAT'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['CFOP'] = pd.to_numeric(planilha['CFOP'])
planilha['CFOP'] = planilha['CFOP'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['UNIDADE'] = planilha['UNIDADE'].str.strip()

def format_number(value):
    return '{:,.2f}'.format(value).replace(',', '.')

planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,"3,000",0.0,...,0.0,0.0,18.7236,Clor Oxibutinina Ems 120ml Xpe,SEM GTIN,UN,30049099,1300201,6161357001371,0.000000e+00
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,"2,000",NaN,...,0.0,0.0,12.4830,Aradois 25mg 30S Biolab Sanus,SEM GTIN,UN,30049069,1300300,6161357001371,0.000000e+00
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Aciclovir Sand 400mg 30S,SEM GTIN,UN,30049069,1300200,6161357001371,0.000000e+00
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Ivermectina 6mg 4S Vitamedic,SEM GTIN,UN,30049059,1300200,6161357001371,0.000000e+00
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Esc D Condor Lilica+Gel D,SEM GTIN,UN,96032100,2005800,6161357001371,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,575708,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Clor Metilfenidato Ems 10mg 60,7896004754161,UN,30049069,1300201,6161357001371,6.161357e+12
1337658,575709,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Entresto 24+26mg 28S Novartis,7896261020269,UN,30049069,1300101,6161357001371,6.161357e+12
1337659,575710,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Klassis Tx+ Serum 30g,7897129304484,UN,33049990,2001500,6161357001371,6.161357e+12
1337660,575711,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Nutriol Lc Hid S/Perf 200ml,3282770205817,UN,33049910,0,6161357001371,6.161357e+12


In [12]:
#query = f'SELECT * FROM tabela_efds_modelo55 WHERE LOJA = {loja}'

#efd = pd.read_sql_query(query, connection)
#efd

In [13]:
planilha = planilha.drop_duplicates()
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,"3,000",0.0,...,0.0,0.0,18.7236,Clor Oxibutinina Ems 120ml Xpe,SEM GTIN,UN,30049099,1300201,6161357001371,0.000000e+00
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,"2,000",NaN,...,0.0,0.0,12.4830,Aradois 25mg 30S Biolab Sanus,SEM GTIN,UN,30049069,1300300,6161357001371,0.000000e+00
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Aciclovir Sand 400mg 30S,SEM GTIN,UN,30049069,1300200,6161357001371,0.000000e+00
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Ivermectina 6mg 4S Vitamedic,SEM GTIN,UN,30049059,1300200,6161357001371,0.000000e+00
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Esc D Condor Lilica+Gel D,SEM GTIN,UN,96032100,2005800,6161357001371,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,575708,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Clor Metilfenidato Ems 10mg 60,7896004754161,UN,30049069,1300201,6161357001371,6.161357e+12
1337658,575709,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Entresto 24+26mg 28S Novartis,7896261020269,UN,30049069,1300101,6161357001371,6.161357e+12
1337659,575710,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Klassis Tx+ Serum 30g,7897129304484,UN,33049990,2001500,6161357001371,6.161357e+12
1337660,575711,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Nutriol Lc Hid S/Perf 200ml,3282770205817,UN,33049910,0,6161357001371,6.161357e+12


In [14]:
planilha['CODIGO_BARRA'] = np.where(pd.notnull(planilha['CODIGO_BARRA']), planilha['CODIGO_BARRA'].astype(str).str.replace(r'\.0$', '', regex=True),
                                   np.nan)
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,"3,000",0.0,...,0.0,0.0,18.7236,Clor Oxibutinina Ems 120ml Xpe,SEM GTIN,UN,30049099,1300201,6161357001371,0.000000e+00
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,"2,000",NaN,...,0.0,0.0,12.4830,Aradois 25mg 30S Biolab Sanus,SEM GTIN,UN,30049069,1300300,6161357001371,0.000000e+00
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Aciclovir Sand 400mg 30S,SEM GTIN,UN,30049069,1300200,6161357001371,0.000000e+00
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Ivermectina 6mg 4S Vitamedic,SEM GTIN,UN,30049059,1300200,6161357001371,0.000000e+00
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Esc D Condor Lilica+Gel D,SEM GTIN,UN,96032100,2005800,6161357001371,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,575708,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Clor Metilfenidato Ems 10mg 60,7896004754161,UN,30049069,1300201,6161357001371,6.161357e+12
1337658,575709,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Entresto 24+26mg 28S Novartis,7896261020269,UN,30049069,1300101,6161357001371,6.161357e+12
1337659,575710,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Klassis Tx+ Serum 30g,7897129304484,UN,33049990,2001500,6161357001371,6.161357e+12
1337660,575711,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,"1,000",NaN,...,0.0,0.0,0.0000,Nutriol Lc Hid S/Perf 200ml,3282770205817,UN,33049910,0,6161357001371,6.161357e+12


In [15]:
mes_ano = []
for date in planilha['DATA']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])

planilha['Data'] = mes_ano
planilha['Data'].unique()

array(['102019', '112019', '122019', '012020', '022020', '032020',
       '042020', '052020', '062020', '072020', '082020', '092020',
       '102020', '112020', '122020'], dtype=object)

In [16]:
planilha.replace('nan', np.nan, inplace=True)

In [17]:
planilha['DATA'].astype(str).iloc[0][8:]

'16'

In [18]:
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,"3,000",0.0,...,0.0,18.7236,Clor Oxibutinina Ems 120ml Xpe,SEM GTIN,UN,30049099,1300201,6161357001371,0.000000e+00,102019
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,"2,000",NaN,...,0.0,12.4830,Aradois 25mg 30S Biolab Sanus,SEM GTIN,UN,30049069,1300300,6161357001371,0.000000e+00,102019
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,"1,000",NaN,...,0.0,0.0000,Aciclovir Sand 400mg 30S,SEM GTIN,UN,30049069,1300200,6161357001371,0.000000e+00,102019
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,"1,000",NaN,...,0.0,0.0000,Ivermectina 6mg 4S Vitamedic,SEM GTIN,UN,30049059,1300200,6161357001371,0.000000e+00,102019
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,"1,000",NaN,...,0.0,0.0000,Esc D Condor Lilica+Gel D,SEM GTIN,UN,96032100,2005800,6161357001371,0.000000e+00,102019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,575708,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,"1,000",NaN,...,0.0,0.0000,Clor Metilfenidato Ems 10mg 60,7896004754161,UN,30049069,1300201,6161357001371,6.161357e+12,122020
1337658,575709,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,"1,000",NaN,...,0.0,0.0000,Entresto 24+26mg 28S Novartis,7896261020269,UN,30049069,1300101,6161357001371,6.161357e+12,122020
1337659,575710,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,"1,000",NaN,...,0.0,0.0000,Klassis Tx+ Serum 30g,7897129304484,UN,33049990,2001500,6161357001371,6.161357e+12,122020
1337660,575711,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,"1,000",NaN,...,0.0,0.0000,Nutriol Lc Hid S/Perf 200ml,3282770205817,UN,33049910,0,6161357001371,6.161357e+12,122020


In [19]:
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,"3,000",0.0,...,0.0,18.7236,Clor Oxibutinina Ems 120ml Xpe,SEM GTIN,UN,30049099,1300201,6161357001371,0.000000e+00,102019
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,"2,000",NaN,...,0.0,12.4830,Aradois 25mg 30S Biolab Sanus,SEM GTIN,UN,30049069,1300300,6161357001371,0.000000e+00,102019
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,"1,000",NaN,...,0.0,0.0000,Aciclovir Sand 400mg 30S,SEM GTIN,UN,30049069,1300200,6161357001371,0.000000e+00,102019
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,"1,000",NaN,...,0.0,0.0000,Ivermectina 6mg 4S Vitamedic,SEM GTIN,UN,30049059,1300200,6161357001371,0.000000e+00,102019
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,"1,000",NaN,...,0.0,0.0000,Esc D Condor Lilica+Gel D,SEM GTIN,UN,96032100,2005800,6161357001371,0.000000e+00,102019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,575708,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,"1,000",NaN,...,0.0,0.0000,Clor Metilfenidato Ems 10mg 60,7896004754161,UN,30049069,1300201,6161357001371,6.161357e+12,122020
1337658,575709,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,"1,000",NaN,...,0.0,0.0000,Entresto 24+26mg 28S Novartis,7896261020269,UN,30049069,1300101,6161357001371,6.161357e+12,122020
1337659,575710,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,"1,000",NaN,...,0.0,0.0000,Klassis Tx+ Serum 30g,7897129304484,UN,33049990,2001500,6161357001371,6.161357e+12,122020
1337660,575711,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,"1,000",NaN,...,0.0,0.0000,Nutriol Lc Hid S/Perf 200ml,3282770205817,UN,33049910,0,6161357001371,6.161357e+12,122020


In [20]:
planilha['CEST'] = planilha['CEST'].replace('0', pd.NA)

# Second, add leading '0' to values with length equal to 6
planilha.loc[planilha['CEST'].str.len() == 6, 'CEST'] = '0' + planilha['CEST']

planilha['CEST']

0          1300201
1          1300300
2          1300200
3          1300200
4          2005800
            ...   
1337657    1300201
1337658    1300101
1337659    2001500
1337660       <NA>
1337661    1300100
Name: CEST, Length: 1337662, dtype: object

In [21]:
planilha['N C M'] = planilha['N C M'].apply(lambda x: str(x).zfill(8) if len(str(x)) < 8 else str(x))
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,"3,000",0.0,...,0.0,18.7236,Clor Oxibutinina Ems 120ml Xpe,SEM GTIN,UN,30049099,1300201,6161357001371,0.000000e+00,102019
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,"2,000",NaN,...,0.0,12.4830,Aradois 25mg 30S Biolab Sanus,SEM GTIN,UN,30049069,1300300,6161357001371,0.000000e+00,102019
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,"1,000",NaN,...,0.0,0.0000,Aciclovir Sand 400mg 30S,SEM GTIN,UN,30049069,1300200,6161357001371,0.000000e+00,102019
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,"1,000",NaN,...,0.0,0.0000,Ivermectina 6mg 4S Vitamedic,SEM GTIN,UN,30049059,1300200,6161357001371,0.000000e+00,102019
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,"1,000",NaN,...,0.0,0.0000,Esc D Condor Lilica+Gel D,SEM GTIN,UN,96032100,2005800,6161357001371,0.000000e+00,102019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,575708,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,"1,000",NaN,...,0.0,0.0000,Clor Metilfenidato Ems 10mg 60,7896004754161,UN,30049069,1300201,6161357001371,6.161357e+12,122020
1337658,575709,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,"1,000",NaN,...,0.0,0.0000,Entresto 24+26mg 28S Novartis,7896261020269,UN,30049069,1300101,6161357001371,6.161357e+12,122020
1337659,575710,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,"1,000",NaN,...,0.0,0.0000,Klassis Tx+ Serum 30g,7897129304484,UN,33049990,2001500,6161357001371,6.161357e+12,122020
1337660,575711,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,"1,000",NaN,...,0.0,0.0000,Nutriol Lc Hid S/Perf 200ml,3282770205817,UN,33049910,<NA>,6161357001371,6.161357e+12,122020


In [22]:
planilha.columns

Index(['index', 'CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM', 'IND_OPER',
       'SUB_TIPO', 'QTD_CAT', 'QTD_INI', 'ICMS_INI', 'QTD_ent1_devolv_ent',
       'ICMS_TOT_0', 'ICMS_TOT_ent_unit', 'ULT_ICMS_TOT_ent_unit',
       'ICMS_TOT_1', 'qtd_saida_1_devolv_saida', 'ICMS_SAIDA_UNI',
       'ULT_ICMS_SAIDA_UNI', 'ICMS_SAIDA', 'ICMS_TOT_PCAT', 'VL_CONFR_0',
       'VLR_CONFR_UNIT', 'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'QTD_SALDO',
       'ICMS_TOT_SALDO', 'VLR_RESSARCIMENTO', 'VLR_COMPLEMENTO', 'COD_LEGAL',
       'COD_LEGAL_PCAT', 'VLR_CONFR_PCAT', 'SALDO_FINAL_MES_QTD',
       'SALDO_FINAL_MES_ICMS', 'ALIQUOTA', 'VALOR',
       'Valor Base Cálculo ICMS ST Retido Operação Anterior',
       'Valor Complementar', 'Valor ICMS Substituição Tributária',
       'Valor ICMS Operação', 'ICMS_TOT', 'DESCRICAO', 'CODIGO_BARRA',
       'UNIDADE', 'N C M', 'CEST', 'CNPJ EMITENTE', 'CNPJ DESTINATARIO',
       'Data'],
      dtype='object')

In [23]:
planilha[planilha['CODIGO_BARRA'] == 'SEM GTIN      ']

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,"3,000",0.0,...,0.00,18.7236,Clor Oxibutinina Ems 120ml Xpe,SEM GTIN,UN,30049099,1300201,6161357001371,0.000000e+00,102019
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,"2,000",NaN,...,0.00,12.4830,Aradois 25mg 30S Biolab Sanus,SEM GTIN,UN,30049069,1300300,6161357001371,0.000000e+00,102019
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,"1,000",NaN,...,0.00,0.0000,Aciclovir Sand 400mg 30S,SEM GTIN,UN,30049069,1300200,6161357001371,0.000000e+00,102019
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,"1,000",NaN,...,0.00,0.0000,Ivermectina 6mg 4S Vitamedic,SEM GTIN,UN,30049059,1300200,6161357001371,0.000000e+00,102019
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,"1,000",NaN,...,0.00,0.0000,Esc D Condor Lilica+Gel D,SEM GTIN,UN,96032100,2005800,6161357001371,0.000000e+00,102019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337348,575399,35200106161357001371590003258520839634277364,2020-01-03,5405,3,933713,1,1,"1,000",NaN,...,0.00,0.0000,Azelan Gel Derm 30g Leo Pharma,SEM GTIN,UN,30049029,1300101,6161357001371,6.161357e+12,012020
1337364,575415,35200461940292000137550320003075361102736345,2020-04-04,1403,7,933713,0,1,"2,000",NaN,...,19.81,28.8648,Azelan Cr Derm 30g,SEM GTIN,UN,30049029,1300101,6161357001371,6.161357e+12,042020
1337499,575550,35200306161357001371590000459452582860780042,2020-03-27,5405,1,935502,1,1,"1,000",NaN,...,0.00,0.0000,"Rexulti 0,5mg 30S C1",SEM GTIN,UN,30049069,1300401,6161357001371,6.161357e+12,032020
1337502,575553,35200406161357001371590000460833400229433786,2020-04-12,5405,7,935502,1,1,"1,000",NaN,...,0.00,0.0000,Xigduo Xr 5mg/1000mg 10x6S,SEM GTIN,UN,30049059,1300101,6161357001371,6.161357e+12,042020


In [24]:
planilha['CODIGO_BARRA'] = planilha['CODIGO_BARRA'].replace('SEM GTIN      ', np.nan)
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,0,35191027849963000110550010005851251005832739,2019-10-16,1403,6,18,0,1,"3,000",0.0,...,0.0,18.7236,Clor Oxibutinina Ems 120ml Xpe,NaN,UN,30049099,1300201,6161357001371,0.000000e+00,102019
1,1,35191027849963000110550010005864571005846114,2019-10-17,1403,6,18,0,1,"2,000",NaN,...,0.0,12.4830,Aradois 25mg 30S Biolab Sanus,NaN,UN,30049069,1300300,6161357001371,0.000000e+00,102019
2,2,35191006161357001371590000459452376749653022,2019-10-22,5405,2,18,1,1,"1,000",NaN,...,0.0,0.0000,Aciclovir Sand 400mg 30S,NaN,UN,30049069,1300200,6161357001371,0.000000e+00,102019
3,3,35191006161357001371590000459452376749653022,2019-10-22,5405,3,18,1,1,"1,000",NaN,...,0.0,0.0000,Ivermectina 6mg 4S Vitamedic,NaN,UN,30049059,1300200,6161357001371,0.000000e+00,102019
4,4,35191006161357001371590000459452376749653022,2019-10-22,5405,4,18,1,1,"1,000",NaN,...,0.0,0.0000,Esc D Condor Lilica+Gel D,NaN,UN,96032100,2005800,6161357001371,0.000000e+00,102019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,575708,35201206161357001371590000459452944477807768,2020-12-02,5405,2,935916,1,1,"1,000",NaN,...,0.0,0.0000,Clor Metilfenidato Ems 10mg 60,7896004754161,UN,30049069,1300201,6161357001371,6.161357e+12,122020
1337658,575709,35201206161357001371590000459452944477807768,2020-12-02,5405,3,935916,1,1,"1,000",NaN,...,0.0,0.0000,Entresto 24+26mg 28S Novartis,7896261020269,UN,30049069,1300101,6161357001371,6.161357e+12,122020
1337659,575710,35201206161357001371590000162453848713824403,2020-12-07,5405,18,935916,1,1,"1,000",NaN,...,0.0,0.0000,Klassis Tx+ Serum 30g,7897129304484,UN,33049990,2001500,6161357001371,6.161357e+12,122020
1337660,575711,35201206161357001371590000162453848713824403,2020-12-07,5405,19,935916,1,1,"1,000",NaN,...,0.0,0.0000,Nutriol Lc Hid S/Perf 200ml,3282770205817,UN,33049910,<NA>,6161357001371,6.161357e+12,122020


In [25]:
list(planilha['CODIGO_BARRA'].unique())

[nan,
 '7891268402025 ',
 '7891142200181 ',
 '7899547524772 ',
 '7896006209690 ',
 '7896253902146 ',
 '7891268107609 ',
 '7896641805776 ',
 '7891268107500 ',
 '7896382706240 ',
 '7891317000714 ',
 '7896026301428 ',
 '7894916550048 ',
 '7896070600607 ',
 '7897595619372 ',
 '7898148291120 ',
 '7896004716732 ',
 '7896212424269 ',
 '7896006200994 ',
 '7896112401582 ',
 '7896026304719 ',
 '7896261020269 ',
 '7896226503721 ',
 '7891268106503 ',
 '7899706124829 ',
 '7896548114612 ',
 '7898148297542 ',
 '7891317476526 ',
 '7896181925224 ',
 '7891268118070 ',
 '7896676416879 ',
 '0789126840652 ',
 '7891317193164 ',
 '7896112140931 ',
 '7898216371037 ',
 '7894916143400 ',
 '7891317001070 ',
 '7896658023361 ',
 '7896261010222 ',
 '7891800197518 ',
 '7891317191740 ',
 '7891010504755 ',
 '7896004710860 ',
 '7899674017079 ',
 '7891134004018 ',
 '7896255767149 ',
 '7896012878859 ',
 '7896641808630 ',
 '7897595901323 ',
 '7896112115151 ',
 '7894916148252 ',
 '7891142115492 ',
 '7897930760424 ',
 '7891

In [26]:
# Geração do dataset base para geração da 1100

cols_1100 = ['CHV_DOC', 'DATA', 'NUM_ITEM', 'IND_OPER', 'COD_ITEM', 'CFOP','QTD_CAT', 'ICMS_TOT_PCAT', 'VLR_CONFR_PCAT', 'COD_LEGAL_PCAT']

df_1100 = planilha.copy()
df_1100 = df_1100[cols_1100]
df_1100['Data'] = df_1100['DATA']
df_1100['DATA'] = df_1100['DATA'].astype(str)
df_1100['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%d%m%Y') for x in df_1100['DATA']]
#df_1100.rename(coluns={'Número Item': 'NUM_ITEM'}, inplace=True)


df_1100['QTD_FIN'] = df_1100['QTD_CAT']
df_1100['COD_REG'] = ['1100']*df_1100.shape[0]
df_1100 = df_1100[['COD_REG','CHV_DOC','Data','DATA','NUM_ITEM','IND_OPER','COD_ITEM','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191027849963000110550010005851251005832739,2019-10-16,16102019,6,0,18,1403,"3,000",18.7236,NaN,NaN
1,1100,35191027849963000110550010005864571005846114,2019-10-17,17102019,6,0,18,1403,"2,000",12.4830,NaN,NaN
2,1100,35191006161357001371590000459452376749653022,2019-10-22,22102019,2,1,18,5405,"1,000",NaN,6.244000,1
3,1100,35191006161357001371590000459452376749653022,2019-10-22,22102019,3,1,18,5405,"1,000",NaN,6.245000,1
4,1100,35191006161357001371590000459452376749653022,2019-10-22,22102019,4,1,18,5405,"1,000",NaN,6.246667,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,1100,35201206161357001371590000459452944477807768,2020-12-02,02122020,2,1,935916,5405,"1,000",NaN,17.871644,1
1337658,1100,35201206161357001371590000459452944477807768,2020-12-02,02122020,3,1,935916,5405,"1,000",NaN,17.871655,1
1337659,1100,35201206161357001371590000162453848713824403,2020-12-07,07122020,18,1,935916,5405,"1,000",NaN,17.871667,1
1337660,1100,35201206161357001371590000162453848713824403,2020-12-07,07122020,19,1,935916,5405,"1,000",NaN,17.871678,1


In [27]:
df_1100['Data'] = df_1100['Data'].astype(str)
df_1100['Data'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m%Y') for x in df_1100['Data']]
df_1100['Data']

0          102019
1          102019
2          102019
3          102019
4          102019
            ...  
1337657    122020
1337658    122020
1337659    122020
1337660    122020
1337661    122020
Name: Data, Length: 1337662, dtype: object

In [28]:
df_1100.dtypes

COD_REG            object
CHV_DOC            object
Data               object
DATA               object
NUM_ITEM            int64
IND_OPER            int64
COD_ITEM            int64
CFOP               object
QTD_FIN            object
ICMS_TOT_PCAT     float64
VLR_CONFR_PCAT    float64
COD_LEGAL_PCAT     object
dtype: object

In [29]:
df_1100['QTD_FIN'] = df_1100['QTD_FIN'].replace('nan', np.nan)
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191027849963000110550010005851251005832739,102019,16102019,6,0,18,1403,"3,000",18.7236,NaN,NaN
1,1100,35191027849963000110550010005864571005846114,102019,17102019,6,0,18,1403,"2,000",12.4830,NaN,NaN
2,1100,35191006161357001371590000459452376749653022,102019,22102019,2,1,18,5405,"1,000",NaN,6.244000,1
3,1100,35191006161357001371590000459452376749653022,102019,22102019,3,1,18,5405,"1,000",NaN,6.245000,1
4,1100,35191006161357001371590000459452376749653022,102019,22102019,4,1,18,5405,"1,000",NaN,6.246667,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1337657,1100,35201206161357001371590000459452944477807768,122020,02122020,2,1,935916,5405,"1,000",NaN,17.871644,1
1337658,1100,35201206161357001371590000459452944477807768,122020,02122020,3,1,935916,5405,"1,000",NaN,17.871655,1
1337659,1100,35201206161357001371590000162453848713824403,122020,07122020,18,1,935916,5405,"1,000",NaN,17.871667,1
1337660,1100,35201206161357001371590000162453848713824403,122020,07122020,19,1,935916,5405,"1,000",NaN,17.871678,1


In [30]:
# Leitura do arquivo da 1050

arquivo_1050 = pd.read_excel(f'Loja_{loja}/1050_loja_{loja}_v1.xlsx', dtype=str)

In [31]:
mes_ano = []
for date in arquivo_1050['REF']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])
    
arquivo_1050['DATA'] = [datetime.strptime(x, '%Y%m').strftime('%m%Y') for x in arquivo_1050['REF']]
arquivo_1050

,Unnamed: 0,REF,COD_ITEM,QTD_INI,ICMS_INI,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,DATA
0,0,201910,18,0,0,8,49.94,102019
1,1,201911,18,8,49.94,9,56.91,112019
2,2,201912,18,9,56.91,11,69.29,122019
3,3,202001,18,11,69.29,6,37.98,012020
4,4,202002,18,6,37.98,7,34.19,022020
...,...,...,...,...,...,...,...,...
124630,124630,202008,935916,192,791.04,192,791.04,082020
124631,124631,202009,935916,192,791.04,192,791.04,092020
124632,124632,202010,935916,0,0,0,0,102020
124633,124633,202011,935916,0,0,6,107.23,112020


In [32]:
for date in planilha['Data'].unique():
    print(date)

102019
112019
122019
012020
022020
032020
042020
052020
062020
072020
082020
092020
102020
112020
122020


In [33]:
# Definição de uma função para exportar os
# dataframes de cada documentação em um
# arquivo .txt com a formatação desejada

def dataframe_to_txt_with_crlf(data, path):
    with open(path, 'a', newline='', encoding='utf-8') as f:
        for index, row in data.iterrows():
            row_str = '|'.join(map(lambda x: '' if pd.isna(x) else str(x), row)) + '\r\n'
            f.write(row_str)

In [34]:
def format_number(value):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Set your desired locale
    formatted_value = locale.format('%.2f', value, grouping=True)
    formatted_value = str(formatted_value).replace('.', '')
    return formatted_value

In [35]:
df_0150 = pd.read_excel('registros_0000_0150.xlsx', sheet_name='0150', dtype=str)
df_0150 = df_0150[['LOJA', 'REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'COD_MUN', 'IE', 'DATA']]
df_0150

,LOJA,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,Loja 191,0150,CLI000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,NaN,3550308,114031767115,20190901
1,Loja 191,0150,CLI000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,NaN,3550308,114031767115,20191001
2,Loja 191,0150,CLI000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,NaN,3550308,114031767115,20190901
3,Loja 191,0150,CLI000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,NaN,3550308,114031767115,20191001
4,Loja 191,0150,CLI751997818,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265005716,NaN,3534401,120211912112,20200601
...,...,...,...,...,...,...,...,...,...,...
139692,Loja 98,0150,FOR753010724,MARLENE ZANON,1058,NaN,02250967814,3550308,NaN,20201201
139693,Loja 98,0150,FOR753012250,MARIA NUCELITA DE SOUSA,1058,NaN,32907008870,3550308,NaN,20201201
139694,Loja 98,0150,FOR753012458,NATAL DE ALMEIDA SOUSA,1058,NaN,00741414813,3550308,NaN,20201201
139695,Loja 98,0150,FOR753013211,CARLOS ROBERTO JUVENCIO,1058,NaN,02017356875,3550308,NaN,20201201


In [36]:
df_0150 = df_0150[df_0150['LOJA'] == f'Loja {loja}']
if df_0150.shape[0] == 0:
        print(f'Loja {loja} não possui 0150. Favor atualizar arquivo')
        sys.exit()
df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
df_0150['CNPJ'] = df_0150['CNPJ'].replace('', np.nan)
df_0150['IE'] = df_0150['IE'].replace('', np.nan)
df_0150['COD_PART'] = df_0150['COD_PART'].str[3:]
df_0150

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\1599971897.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\1599971897.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\1599971897.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,LOJA,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
48496,Loja 28,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,3550308,116939809113,092019
48497,Loja 28,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,3550308,149399539118,092019
48498,Loja 28,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,3550308,147552748114,092019
48499,Loja 28,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,3550308,145389352117,092019
48500,Loja 28,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,3550308,142004487112,092019
...,...,...,...,...,...,...,...,...,...,...
50714,Loja 28,0150,753013954,Consumidor final,1058,NaN,06936383890,3550308,NaN,122020
50715,Loja 28,0150,753013955,Consumidor final,1058,NaN,26284700806,3550308,NaN,122020
50716,Loja 28,0150,753013956,Consumidor final,1058,NaN,34378069806,3550308,NaN,122020
50717,Loja 28,0150,753014099,Consumidor final,1058,NaN,30484107836,3550308,NaN,122020


In [37]:
df_0150 = df_0150[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'IE', 'COD_MUN', 'DATA']]
df_0150 = df_0150[df_0150['CNPJ'].notnull()]
df_0150 = df_0150.drop_duplicates()
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
48496,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,092019
48497,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,092019
48498,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,092019
48499,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,092019
48500,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,092019
...,...,...,...,...,...,...,...,...,...
50630,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
50632,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,122020
50647,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,206165226110,3505708,122020
50649,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,122020


In [38]:
mask_duplicates = df_0150[df_0150['DATA'] == '112019'].duplicated(subset='CNPJ', keep='first')
# Invert the mask to keep the first occurrence of each unique 'CNPJ' value
mask_to_keep = ~mask_duplicates

# Apply the mask to the DataFrame to keep the desired rows for the current 'PERIODO'
filtered_df = df_0150[(df_0150['DATA'] == '112019') | mask_to_keep]
filtered_df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
48794,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,112019
48795,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,112019
48796,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,112019
48797,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,112019
48798,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,112019
48799,0150,000156510,Prevent senior private op.de saude ltda,1058,00461479004150,NaN,NaN,3550308,112019
48800,0150,000156512,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001992,NaN,145503067113,3550308,112019
48801,0150,000156514,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003006,NaN,145389380113,3550308,112019
48802,0150,000156521,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001305,NaN,145389361118,3550308,112019
48803,0150,000156531,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003421,NaN,635638841110,3548708,112019


In [39]:
filtered_df['CNPJ'].unique()

array(['06161357000804', '06161357001029', '06161357001371',
       '00461479001054', '00461479003189', '00461479004150',
       '00461479001992', '00461479003006', '00461479001305',
       '00461479003421', '60958238000156', '00461479007176',
       '00461479004584', '00461479000910', '00461479001216',
       '00461479005122', '27849963000110', '00461479005475',
       '00461479002379', '25216400000160', '00461479001720',
       '00461479006870', '00461479005041', '00461479007257',
       '07463596000105', '00461479008490', '11195501000197',
       '61940292000137', '23276873000127', '05651966000455',
       '44463156000184', '01206820000520', '45453214001042',
       '97554090000102', '11806723000794', '10328983000143',
       '61695227000193', '07259917000154', '33918452000106',
       '61068276001682'], dtype=object)

In [40]:
df_0150_final = pd.DataFrame()

# Get the unique 'PERIODO' values
unique_periodos = df_0150['DATA'].unique()

# Iterate through unique 'PERIODO' values
for periodo in unique_periodos:
    # Filter the DataFrame for the current 'PERIODO' value
    filtered_df = df_0150[df_0150['DATA'] == periodo]
    
    # Drop duplicates in the 'CNPJ' column for the current 'PERIODO'
    filtered_df = filtered_df.drop_duplicates(subset='CNPJ', keep='first')
    
    # Append the filtered results to the final DataFrame
    df_0150_final = df_0150_final.append(filtered_df, ignore_index=True)

# Reset the index of the final DataFrame
df_0150_final.reset_index(drop=True, inplace=True)
df_0150_final = df_0150_final[df_0150_final['REG'] == '0150']
df_0150_final

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\575281542.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_0150_final = df_0150_final.append(filtered_df, ignore_index=True)


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,092019
1,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,092019
2,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,092019
3,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,092019
4,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,092019
...,...,...,...,...,...,...,...,...,...
576,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
577,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,122020
578,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,206165226110,3505708,122020
579,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,122020


In [41]:
df_0150_final = df_0150_final.drop_duplicates()
df_0150_final[df_0150_final['IE'].notnull()]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,092019
1,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,092019
2,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,092019
3,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,092019
4,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,092019
...,...,...,...,...,...,...,...,...,...
576,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
577,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,122020
578,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,206165226110,3505708,122020
579,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,122020


In [42]:
df_cnpj_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']
for date in dates:
    for cnpj in df_0150_final[df_0150_final['DATA'] == date]['CNPJ'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]
            df_cnpj_repetido = pd.concat([df_cnpj_repetido, df])
            
df_cnpj_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [43]:
df_ie_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']
for date in dates:
    for ie in df_0150_final[df_0150_final['DATA'] == date]['IE'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]
            df_ie_repetido = pd.concat([df_ie_repetido, df])
            
df_ie_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
329,0150,751872165,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,052020
333,0150,751992484,EDIFICIO PIAZZA DEL SOLE,1058,11511935000159,NaN,234086958110,3550308,052020


In [44]:
# Iterate through each file in the folder
df_0000 = pd.read_excel('registros_0000_0150.xlsx', sheet_name='0000', dtype=str)

df_0000['DATA'] = df_0000['DATA'].astype(str).str.slice(0, 8)
df_0000['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0000['DATA']]
df_0000['COD_VER'] = ['01'] * df_0000.shape[0]
df_0000['COD_FIN'] = ['00'] * df_0000.shape[0]
df_0000 = df_0000[['LOJA','REG', 'DATA', 'NOME', 'CNPJ', 'IE',
        'COD_MUN', 'COD_VER', 'COD_FIN']]

df_0000

,LOJA,REG,DATA,NOME,CNPJ,IE,COD_MUN,COD_VER,COD_FIN
0,Loja 191,0000,092019,Master Formula Farmacia de Manipulacao LTDA,71605265002105,543035938119,3539509,01,00
1,Loja 191,0000,102019,Master Formula Farmacia de Manipulacao LTDA,71605265002105,543035938119,3539509,01,00
2,Loja 191,0000,112019,Master Formula Farmacia de Manipulacao LTDA,71605265002105,543035938119,3539509,01,00
3,Loja 191,0000,122019,Master Formula Farmacia de Manipulacao LTDA,71605265002105,543035938119,3539509,01,00
4,Loja 191,0000,012020,Master Formula Farmacia de Manipulacao LTDA,71605265002105,543035938119,3539509,01,00
...,...,...,...,...,...,...,...,...,...
2777,Loja 98,0000,082020,Master Formula Farmacia de Manipulação LTDA,71605265037405,136948812117,3550308,01,00
2778,Loja 98,0000,092020,Master Formula Farmacia de Manipulação LTDA,71605265037405,136948812117,3550308,01,00
2779,Loja 98,0000,102020,Master Formula Farmacia de Manipulação LTDA,71605265037405,136948812117,3550308,01,00
2780,Loja 98,0000,112020,Master Formula Farmacia de Manipulação LTDA,71605265037405,136948812117,3550308,01,00


In [45]:
for date in planilha['Data'].unique():
    print(date)

102019
112019
122019
012020
022020
032020
042020
052020
062020
072020
082020
092020
102020
112020
122020


In [46]:
df_0150_final['COD_MUN'] = pd.to_numeric(df_0150_final['COD_MUN'])
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,092019
1,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,092019
2,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,092019
3,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,092019
4,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,092019
...,...,...,...,...,...,...,...,...,...
576,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
577,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,122020
578,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,206165226110,3505708,122020
579,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,122020


In [47]:
def fill_missing_ie(group):
    non_null_ie = group.dropna()['IE']
    if not non_null_ie.empty:
        group['IE'] = non_null_ie.iloc[0]
    elif group['IE'].isnull().all():
        group['IE'] = np.nan
    return group

# Group by 'CNPJ' and apply the custom function
df_0150_final_new = df_0150_final.groupby('CNPJ', group_keys=False).apply(fill_missing_ie)

df_0150_final_new

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,092019
1,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,092019
2,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,092019
3,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,092019
4,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,092019
...,...,...,...,...,...,...,...,...,...
576,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
577,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,122020
578,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,206165226110,3505708,122020
579,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,122020


In [48]:
df_0150_final = df_0150_final[df_0150_final['IE'].notnull()]
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,092019
1,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,092019
2,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,092019
3,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,092019
4,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,092019
...,...,...,...,...,...,...,...,...,...
576,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
577,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,122020
578,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,206165226110,3505708,122020
579,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,122020


In [49]:
df_0150_final.loc[(df_0150_final['CNPJ'].notnull()) & (df_0150_final['IE'].isnull())]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [50]:
df_0150_final['CNPJ'] = df_0150_final['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))
df_0150_final

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\2152152490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150_final['CNPJ'] = df_0150_final['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,092019
1,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,092019
2,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,092019
3,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,092019
4,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,092019
...,...,...,...,...,...,...,...,...,...
576,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
577,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,122020
578,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,206165226110,3505708,122020
579,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,122020


In [51]:
# Função para verificar o dígito verificador de um CNPJ
def verifica_digito_verificador(cnpj):
    cnpj = [int(d) for d in str(cnpj) if d.isdigit()]  # Converte o CNPJ em uma lista de números inteiros
    if len(cnpj) != 14:
        return False  # CNPJ deve ter 14 dígitos

    # Calcula o primeiro dígito verificador
    soma = 0
    peso = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    for i in range(12):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador1 = 0
    else:
        digito_verificador1 = 11 - resto

    # Calcula o segundo dígito verificador
    soma = 0
    peso.insert(0, 6)
    for i in range(13):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador2 = 0
    else:
        digito_verificador2 = 11 - resto

    # Verifica se os dígitos verificadores são iguais aos dígitos originais
    return cnpj[-2] == digito_verificador1 and cnpj[-1] == digito_verificador2

# Exemplo de DataFrame
df = df_0150_final.copy()

# Aplica a função verifica_digito_verificador à coluna 'CNPJ' do DataFrame
df['Dígito_Verificador_Válido'] = df['CNPJ'].apply(verifica_digito_verificador)

# Exibe o DataFrame com a nova coluna
df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA,Dígito_Verificador_Válido
0,0150,000000309,Conde Neto & Cia Ltda,1058,06161357000804,NaN,116939809113,3550308,092019,True
1,0150,000000311,Conde Neto & Cia Ltda,1058,06161357001029,NaN,149399539118,3550308,092019,True
2,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,092019,True
3,0150,000156500,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001054,NaN,145389352117,3550308,092019,True
4,0150,000156504,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,092019,True
...,...,...,...,...,...,...,...,...,...,...
576,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020,True
577,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,234086958110,3508504,122020,True
578,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,206165226110,3505708,122020,True
579,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,122020,True


In [52]:
df['Dígito_Verificador_Válido'].unique()

array([ True])

In [53]:
unique_cod_items = planilha[planilha['Data'] == '102019']['COD_ITEM'].unique()
arquivo_1050['COD_ITEM'] = arquivo_1050['COD_ITEM'].astype(int)
filtered = arquivo_1050[arquivo_1050['DATA'] == '102019'][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
filtered

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\641789374.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == '102019'][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]


,Unnamed: 0,REF,COD_ITEM,QTD_INI,ICMS_INI,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,DATA
0,0,201910,18,0,0,8,49.94,102019
15,15,201910,26,10,0,10,90.53,102019
30,30,201910,34,0,0,8,39.21,102019
45,45,201910,59,2,0,3,18.1,102019
60,60,201910,75,3,0,37,82.51,102019
...,...,...,...,...,...,...,...,...
124485,124485,201910,933713,1,10.70234747425896,2,30.11,102019
124515,124515,201910,933754,1,20.29713865846505,2,52.41,102019
124560,124560,201910,935502,3,3.374317145377135,11,13.76,102019
124575,124575,201910,935510,1,28.43367012785115,1,32.66,102019


In [55]:
# Geração dos TXTs da CAT

for date in planilha['Data'].unique():
#for date in ['112019']:
    # Geração da 0000
    df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
    df_0000_['IE'] = df_0000_['IE'].str.replace('-', '')
    df_0000_ = df_0000_.drop('LOJA', axis=1)
    
    # Geração da 0150

    # df_0150 has originally 8 columns
    df_0150_ = df_0150_final[df_0150_final['DATA'] == date]
    if df_0150_.shape[0] == 0:
        print(f'Loja {loja} não possui 0150. Favor atualizar arquivo')
        sys.exit()
    df_0150_ = df_0150_[df_0150_['CNPJ'].notnull()]
    df_0150_['IE'] = df_0150_['IE'].str.replace('-', '')
    df_0150_ = df_0150_[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                'IE', 'COD_MUN']]
    
    # Geração da 0200
    df_0200 = planilha[planilha['Data'] == date][['COD_ITEM','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']].drop_duplicates()
    df_0200['COD_REG'] = ['0200']*(df_0200.shape[0])
    df_0200 = df_0200[['COD_REG', 'COD_ITEM','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']]
    df_0200['ALIQUOTA'] = df_0200['ALIQUOTA'].astype(float).astype(int)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].astype(str)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].replace('nan', np.nan)
    df_0200.drop_duplicates(subset='COD_ITEM', keep='first', inplace=True)
    
    # Geração da 1050
    unique_cod_items = planilha[planilha['Data'] == date]['COD_ITEM'].unique()
    arquivo_1050['COD_ITEM'] = arquivo_1050['COD_ITEM'].astype(int)
    filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
    filtered

    df_1050 = filtered[['COD_ITEM','QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    
    df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
    df_1050['QTD_INI'] = df_1050['QTD_INI'].astype(float).astype(int).astype(str) + ',000'
    df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
    df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
    df_1050['SALDO_FINAL_MES_ICMS'] = df_1050['SALDO_FINAL_MES_ICMS'].astype(float).map(format_number)
    
    df_1050 = df_1050[['COD_REG', 'COD_ITEM', 'QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    df_1050.rename(columns={'COD_REG': 'REG'}, inplace=True)
    
    # Geração da 1100
    df_1100_ = df_1100[df_1100['Data'] == date]
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','COD_ITEM','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
    
    # Geração do .txt da 0000
    dataframe_to_txt_with_crlf(df_0000_, f'Loja_{loja}/CAT_Loja_{loja}/teste/0000_{date}_v1.txt')
    
    # Geração do .txt da 0150
    dataframe_to_txt_with_crlf(df_0150_, f'Loja_{loja}/CAT_Loja_{loja}/teste/0150_{date}_v1.txt')
    
    # Geração do .txt da 0200
    dataframe_to_txt_with_crlf(df_0200, f'Loja_{loja}/CAT_Loja_{loja}/teste/0200_{date}_v1.txt')
    
    # Geração do .txt da 1050
    dataframe_to_txt_with_crlf(df_1050, f'Loja_{loja}/CAT_Loja_{loja}/teste/1050_{date}_v1.txt')
    
    # Geração do .txt da 1100
    dataframe_to_txt_with_crlf(df_1100_, f'Loja_{loja}/CAT_Loja_{loja}/teste/1100_{date}_v1.txt')
    
    # Geração de um arquivo .txt único,
    # contendo todas as informações das documentações
    # da 0000, 0150, 0200, 1050 e 1100
    # dentro do formato correto

    read_files = glob.glob(f"Loja_{loja}/CAT_Loja_{loja}/teste/*{date}_v1.txt")
    
    with open(f"Loja_{loja}/CAT_Loja_{loja}/cat_loja_{loja}_{date}_v1.txt", "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:40: SettingWithCopyWarning: 
A value is trying to be set

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:53: SettingWithCopyWarning: 
A value is 

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['SALDO_FINAL_MES_ICMS'] = df_1050['SALDO_FINAL_MES_ICMS'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\1795614597.py:3: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  formatted_value = locale.format('%.2f', value, grouping=True)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:43: SettingWithCopyWarning: 
A val

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:40: SettingWithCopyWarning: 
A value is trying to be set

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:53: SettingWithCopyWarning: 
A value is 

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\1795614597.py:3: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  formatted_value = locale.format('%.2f', value, grouping=True)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:43: SettingWithCopyWarning: 
A val

C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_18964\3608821322.py:40: SettingWithCopyWarning: 
A value is trying to be set